In [156]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src')


import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
"""
PREP DATA
"""

from get_data import load_history, load_current, combine_history, process_y, load_bios, load_team_data
from model_training import make_predictions

# load_history()
# load_current()
# combine_history()
# load_team_data()
# load_bios()
# process_y()
# make_predictions()

import yahoo_utils
# yahoo_utils.refresh_players()
# yahoo_utils.refresh_player_games()
# yahoo_utils.refresh_teams()

In [603]:
from model_training import load_predictions
import lineup_utils
import yahoo_utils

        
class TeamAnalytics():
    
    def __init__(self, mode='season', date=None, week=None, team_key=None):
        self.mode = mode
        
        if mode == 'season':
            self.date = pd.Timestamp.now(tz='America/Los_Angeles').date()
            self.matchup = None
        
        elif mode == 'week':
            self.week = week
            self.matchup = yahoo_utils.get_matchup(date=date, week=week)
            self.date = self.matchup['start'] - pd.Timedelta('1d')
            
        else:
            raise ValueError('Invalid value for mode')

        self.players = yahoo_utils.load_players(self.matchup)
        self.preds, self.player_stds = load_predictions()
        self.player_games = yahoo_utils.load_player_games()
        
        if self.mode == 'week':
            date_filter = (self.player_games.date.dt.date <= self.matchup['end']) & (self.player_games.date.dt.date >= self.matchup['start'])
            self.player_games = self.player_games.loc[date_filter]
        
        self.team_key = team_key
        if self.team_key is not None:
            self.current_team = self.players[self.players.team_key == team_key].index.tolist()
            opp_lineup = self.players[self.players.team_key == self.matchup['opponent']].index.tolist()
            self.preds = self.preds.loc[self.current_team + opp_lineup]
            self.player_games = self.player_games.loc[self.current_team + opp_lineup]
        
        # could be populated instead of hardcoded
        self.cats = ['g','a','sog','fow','hit','block','pim','plusmin','ppp', 'ga','win','so','save']
        
        
        self.selected_team = []        
        self.n_games_lineup = 0
        self.own_totals = pd.Series({c:0 for c in self.cats})
        
        self.rest_games = None
        self.lineup_preds = None
        self.rel_lineup_preds = None
        self.get_lineup_preds()
    
    def get_lineup_preds(self):
        self.rest_games = lineup_utils.get_rest_of_period_games(self.date, self.player_games.reset_index(), self.selected_team, self.players, self.preds)
        stats_available = rest_games[rest_games.index.isin(self.preds.index) & ~rest_games.index.isin(self.selected_team)].index
        self.lineup_preds = self.preds.loc[stats_available, self.cats].apply(lambda x: x * self.rest_games[stats_available])
        self.lineup_preds['ga'] = self.preds.loc[stats_available, 'ga']
        self.lineup_preds['games_left'] = rest_games.loc[stats_available]
        self.lineup_preds = self.lineup_preds.join(self.players[['available']], how='left')
        
        if self.mode == 'week':
            comparison = [p for p in self.players.loc[self.players.opp_lineup].index if p in self.lineup_preds.index]
        else:
            comparison = [p for p in self.players.loc[self.players.is_rostered].index if p in self.lineup_preds.index]
      

        # standard deviation that takes into account individual players' uncertainty
        std_factor =  ( (self.preds.loc[comparison, self.cats].var()) + (self.player_stds / (17**0.5))**2 ) ** 0.5
        lineup_len_divisors = (self.lineup_preds.notna() + self.preds.loc[self.selected_team, self.cats].count()).apply(lambda x: x * (self.n_games_lineup + self.rest_games.mean()) / (len(self.selected_team)+1))

        self.rel_lineup_preds = ((self.lineup_preds.fillna(0) + self.own_totals)/lineup_len_divisors - self.preds.loc[comparison, self.cats].mean()) / std_factor
        self.lineup_preds['rank'] = self.rel_lineup_preds.sum(axis=1)
        self.lineup_preds['games_left'] = self.rest_games.loc[stats_available]
        
        return self
    
    
    def select_player(self, selected_player):
        
        self.n_games_lineup += self.rest_games.loc[selected_player]
        self.selected_team.append(selected_player)
        self.own_totals += self.lineup_preds.loc[selected_player, self.cats].astype(float).fillna(0)
        self.own_totals['ga'] = self.preds.loc[self.selected_team, 'ga'].mean()
        
        self.get_lineup_preds()
        return self
    
    def unselect_player(self, selected_player):
        self.selected_team.remove(selected_player)
        self.get_lineup_preds()
        
        self.own_totals -= self.lineup_preds.loc[selected_player, self.cats].astype(float).fillna(0)
        self.own_totals['ga'] = self.preds.loc[self.selected_team, 'ga'].mean()
        
        self.n_games_lineup -= self.rest_games.loc[selected_player]
        return self
    
    def render(self):
        return self.players.join(self.lineup_preds, how='inner').sort_values('rank')
    
    def make_full_selections(self):
        if self.team_key is None:
            team_max_length = 17
        else:
            team_max_length = min(len(self.current_team), 17)
        
        while len(self.selected_team) < team_max_length and len(self.lineup_preds) > 0:
            selection = self.lineup_preds.loc[self.lineup_preds.available, 'rank'].idxmax()
            self.select_player(selection)
            print(f'Selected {self.players.loc[selection]['name']}')
    
    def select_team(self, team_key):
        team_data = TeamAnalytics(mode=self.mode, team_key=team_key, week=self.week)
        team_data.make_full_selections()
        self.selected_team = team_data.selected_team
        self.n_games_lineup = team_data.n_games_lineup
        self.own_totals = team_data.own_totals
        self.get_lineup_preds()
        
    def select_none(self):
        self.selected_team = []
        self.n_games_lineup = 0
        self.own_totals = pd.Series({c:0 for c in self.cats})
        self.get_lineup_preds()
        

In [605]:
full_data = TeamAnalytics(mode='week', date=pd.Timestamp.now().date)
full_data.select_team('453.l.15482.t.3')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): fantasysports.yahooapis.com:443
DEBUG:urllib3.connectionpool:https://fantasysports.yahooapis.com:443 "GET /fantasy/v2/game/453/metadata?format=json HTTP/11" 200 354
DEBUG:urllib3.connectionpool:https://fantasysports.yahooapis.com:443 "GET /fantasy/v2/team/453.l.15482.t.3;out=metadata,stats,standings,roster,draftresults,matchups?format=json HTTP/11" 200 21637
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): fantasysports.yahooapis.com:443
DEBUG:urllib3.connectionpool:https://fantasysports.yahooapis.com:443 "GET /fantasy/v2/game/453/metadata?format=json HTTP/11" 200 352
DEBUG:urllib3.connectionpool:https://fantasysports.yahooapis.com:443 "GET /fantasy/v2/team/453.l.15482.t.3;out=metadata,stats,standings,roster,draftresults,matchups?format=json HTTP/11" 200 21638
Selected John Tavares
Selected Sam Montembeault
Selected Cam Talbot
Selected Tage Thompson
Selected Matt Boldy
Selected Seth Jones
Selected Steven Sta

In [606]:
full_data.matchup

{'week': 1,
 'opponent': '453.l.15482.t.1',
 'start': datetime.date(2024, 10, 4),
 'end': datetime.date(2024, 10, 13)}

In [580]:
full_data = TeamAnalytics(mode='week', week=8, team_key='453.l.15482.t.3')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): fantasysports.yahooapis.com:443
DEBUG:urllib3.connectionpool:https://fantasysports.yahooapis.com:443 "GET /fantasy/v2/game/453/metadata?format=json HTTP/11" 200 354
DEBUG:urllib3.connectionpool:https://fantasysports.yahooapis.com:443 "GET /fantasy/v2/team/453.l.15482.t.3;out=metadata,stats,standings,roster,draftresults,matchups?format=json HTTP/11" 200 21638


In [578]:
full_data.lineup_preds

,g,a,sog,fow,hit,block,pim,plusmin,ppp,ga,win,so,save,games_left,rank
8479420,1.364782,1.757361,9.690710,16.559123,2.683591,1.067987,1.152815,0.440458,0.877156,NaN,NaN,NaN,NaN,3.0,-0.505656
8471675,1.197262,1.841683,8.713537,37.955161,3.544683,1.400703,1.303392,0.051954,0.709436,NaN,NaN,NaN,NaN,3.0,-0.048705
8478009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.310149,1.196281,0.132032,78.428720,4.0,-3.008666
8475660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.336630,1.448996,0.144707,76.958659,4.0,-2.332825
8476906,0.612891,2.156140,8.092205,0.043400,3.168833,3.510747,1.634587,0.476979,1.150526,NaN,NaN,NaN,NaN,4.0,-0.263841
8480830,1.481311,2.052497,12.164158,0.305924,6.336003,1.587585,3.109815,0.802657,1.047943,NaN,NaN,NaN,NaN,4.0,3.895335
8477495,0.486905,1.278153,6.478363,-0.004702,4.773793,5.328292,1.511303,-0.026728,0.432714,NaN,NaN,NaN,NaN,3.0,-3.492382
8484144,0.886969,1.429492,7.866642,6.960987,2.036502,1.620851,1.110614,0.006996,0.527370,NaN,NaN,NaN,NaN,3.0,-4.271961
8481028,0.571483,0.756691,6.700366,8.161831,12.235820,1.303740,4.133773,-0.185956,0.060805,NaN,NaN,NaN,NaN,4.0,-1.786069
8480012,1.327314,1.908884,9.375396,25.742565,5.996763,3.920033,1.661916,0.523973,0.933612,NaN,NaN,NaN,NaN,4.0,3.006787


In [558]:
df = full_data.players.join(full_data.lineup_preds).sort_values('rank', na_position='first')
df[df.available]
full_data.lineup_preds

,g,a,sog,fow,hit,block,pim,plusmin,ppp,ga,win,so,save,games_left,rank
8479420,1.364782,1.757361,9.690710,16.559123,2.683591,1.067987,1.152815,0.440458,0.877156,NaN,NaN,NaN,NaN,3.0,0.0
8475660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.336630,1.448996,0.144707,76.958659,4.0,0.0
8476906,0.612891,2.156140,8.092205,0.043400,3.168833,3.510747,1.634587,0.476979,1.150526,NaN,NaN,NaN,NaN,4.0,0.0
8477495,0.486905,1.278153,6.478363,-0.004702,4.773793,5.328292,1.511303,-0.026728,0.432714,NaN,NaN,NaN,NaN,3.0,0.0
8479292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.344407,1.361501,0.133621,73.997973,4.0,0.0
8474564,1.878346,1.810822,12.236424,18.772898,3.599732,1.627097,1.947687,0.132328,1.263918,NaN,NaN,NaN,NaN,4.0,0.0
8475166,0.884450,0.844392,6.075306,20.061258,3.365075,0.680003,1.141264,0.125703,0.484932,NaN,NaN,NaN,NaN,2.0,0.0
8478469,0.545262,1.984700,9.408400,0.318519,3.819062,6.526900,1.402297,0.795992,0.641121,NaN,NaN,NaN,NaN,4.0,0.0
8477986,0.851397,1.934207,10.829440,0.161752,5.839003,5.161373,2.309644,0.491561,0.771164,NaN,NaN,NaN,NaN,4.0,0.0
8477935,1.233023,1.391608,10.003374,21.999984,8.417815,2.192957,3.047394,0.603501,0.617189,NaN,NaN,NaN,NaN,4.0,0.0


In [ ]:
full_data.matchup

In [532]:
(full_data.player_games.date.dt.date <= full_data.matchup['end']) & (full_data.player_games.date.dt.date >= full_data.matchup['start'])

playerId
8473449.0    False
8473503.0    False
8475722.0    False
8477365.0    False
8477949.0    False
             ...  
8481601.0    False
8481541.0    False
8481740.0    False
8483448.0    False
8484958.0    False
Name: date, Length: 85635, dtype: bool

In [ ]:
full_data

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
full_data.render()